In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import xarray as xr
import numpy as np
#import pscpy

%matplotlib ipympl 
#plt.rcParams['figure.figsize'] = [16, 10]

In [2]:
# dipole far field
dir = "/Users/kai/src/psc/build-arm64"
steps = range(0, 400, 5)
vmax = .0001

# orig dipole near field
dir = "/Users/kai/src/psc/build-arm64-2"
steps = range(0, 800, 5)
vmax = .01

# quadrupole
dir = "/Users/kai/src/psc/build-arm64-2"
steps = range(0, 1000, 20)
vmax = .00005

def open_step(step):
    return xr.open_dataset(f"{dir}/pfd.{step:09d}.bp", #engine='pscadios2',
                         species_names=['e', 'i'])



In [ ]:
datas = {}
for step in steps:
    ds = open_step(step)
    datas[step] = ds.ez_ec.sel(y=0.).T, float(ds.time)
#datas

In [ ]:
def get_data(step):
    return datas[step]

steps = list(steps)

fig, ax = plt.subplots()
step = steps[0]
fld, time = get_data(step)
cax = fld.plot(vmin=-vmax, vmax=vmax, cmap='coolwarm')
ax.set_title(f"step {step} time {time:6.2f}")
ax.set_aspect(1.)

def animate(step):
    fld, time = get_data(step)
    cax.set_array(fld.values.flatten())
    ax.set_title(f"step {step} time {time:6.2f}")

ani = FuncAnimation(
    fig,             # figure
    animate,         # name of the function above
    frames=steps[1:],    # Could also be iterable or list
    interval=200,     # ms between frames
    blit=False
)

plt.show()
#ani

In [5]:
def plot_fields(fldnames, fld_kwargs=None):
    fig, axs = plt.subplots(1, len(fldnames))
    if len(fldnames) == 1: axs = [axs]
    for i, fldname in enumerate(fldnames):
        fld = ds[fldname].sel(y=0)
        if fld_kwargs:
            kwargs = fld_kwargs[i]
        else:
            kwargs = {}
        fld.plot(ax=axs[i], **kwargs)
        axs[i].set_aspect('equal')
    plt.tight_layout()

In [ ]:

plot_fields(['ex_ec', 'ey_ec', 'ez_ec'])
#            fld_kwargs=[{"vmin": -.0065}, {"vmin": -.02}, {}])

In [ ]:
def format_radians_label(float_in):
    # Converts a float value in radians into a
    # string representation of that float
    string_out = str(float_in / (np.pi))+"π"
    
    return string_out

def convert_polar_xticks_to_radians(ax):
    # Converts x-tick labels from degrees to radians
    
    # Get the x-tick positions (returns in radians)
    label_positions = ax.get_xticks()
    
    # Convert to a list since we want to change the type of the elements
    labels = list(label_positions)
    
    # Format each label (edit this function however you'd like)
    labels = [format_radians_label(label) for label in labels]
    
    ax.set_xticklabels(labels)
    
theta = np.linspace(-np.pi, np.pi, 100)

fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
ax.plot(theta, np.sin(theta)**2)
ax.set_rticks([0.25, 0.5, 0.75, 1])
ax.set_theta_zero_location("N")
#convert_polar_xticks_to_radians(ax)

ax.set_title("Radiated dipole power")
plt.show()